<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/%E9%81%BA%E4%BC%9D%E5%AD%90_%E7%B5%9E%E3%82%8A%E8%BE%BC%E3%81%BF_NCBI_%E3%82%B3%E3%83%BC%E3%83%89_1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [9]:
import pandas as pd
from Bio import Entrez
import time

# BiopythonのEntrezモジュールを初期化
Entrez.email = "tarumi.wataru@gmail.com"  # あなたのメールアドレスを設定してください
Entrez.api_key = "d9702839c15b5365a594355c0c6974cc6207"  # あなたのAPIキーを設定してください

# エクセルファイルから遺伝子IDを読み込む
excel_file = "/content/GV_MⅡ_GSE75738.xls.xlsx"  # エクセルファイルのパスを指定してください
N = 3300  # 読み込む行数を設定
gene_df = pd.read_excel(excel_file, nrows=N)
gene_ids = gene_df.iloc[:, 0].tolist()  # 一列目の遺伝子IDを取得

# 検索対象のキーワードを指定
search_keyword = "ligand"  # 検索したいキーワードを指定してください

# 遺伝子情報を取得し、指定したキーワードを含む遺伝子と遺伝子名を抽出
matching_genes = []

# リクエストの間隔を設定
request_interval = 5  # 5秒ごとにリクエストを送信
batch_size = 50  # 一度に処理する遺伝子IDの数

for i in range(0, len(gene_ids), batch_size):
    batch_gene_ids = gene_ids[i:i+batch_size]

    # batch_gene_idsを文字列に変換
    batch_gene_ids = [str(gene_id) for gene_id in batch_gene_ids]

    try:
        # EPostリクエストを使用して遺伝子IDをアップロード
        post_handle = Entrez.epost(db="gene", id=",".join(batch_gene_ids))
        post_result = Entrez.read(post_handle)
        web_history = post_result["WebEnv"]
        query_key = post_result["QueryKey"]
        post_handle.close()

        # EUtilリクエストを使用してデータを取得
        fetch_handle = Entrez.efetch(db="gene", query_key=query_key, WebEnv=web_history, retmode="xml")
        records = Entrez.read(fetch_handle)
        fetch_handle.close()

        # 遺伝子情報からキーワードを検索
        for record in records:
            gene_summary = record.get("Entrezgene_summary", "")
            gene_id = record.get("Entrezgene_trackinfo", {}).get("Gene-track_geneid", "N/A")
            gene_name = record.get("Entrezgene_gene", "N/A")

            if search_keyword in gene_summary:
                matching_genes.append((gene_id, gene_name))
    except Exception as e:
        print(f"エラーが発生しました: {e}")

    # リクエストの間隔を設ける
    time.sleep(request_interval)

# 検索結果をCSVファイルに保存
if matching_genes:
    result_df = pd.DataFrame(matching_genes, columns=["Gene ID", "Gene Name"])
    result_csv = "matching_genes.csv"
    result_df.to_csv(result_csv, index=False)
    print(f"以下の遺伝子がキーワードを含んでいます。結果は'{result_csv}'に保存されました:")
    print(result_df)
else:
    print("指定したキーワードを含む遺伝子は見つかりませんでした。")

エラーが発生しました: HTTP Error 400: Bad Request
エラーが発生しました: IncompleteRead(1449 bytes read)
エラーが発生しました: HTTP Error 400: Bad Request
以下の遺伝子がキーワードを含んでいます。結果は'matching_genes.csv'に保存されました:
   Gene ID                                          Gene Name
0      N/A  {'Gene-ref': {'Gene-ref_locus': 'GABRB3', 'Gen...
1      N/A  {'Gene-ref': {'Gene-ref_locus': 'TNFRSF1A', 'G...
2      N/A  {'Gene-ref': {'Gene-ref_locus': 'EFNB1', 'Gene...
3      N/A  {'Gene-ref': {'Gene-ref_locus': 'S1PR1', 'Gene...
4      N/A  {'Gene-ref': {'Gene-ref_locus': 'CYGB', 'Gene-...
..     ...                                                ...
74     N/A  {'Gene-ref': {'Gene-ref_locus': 'NR3C2', 'Gene...
75     N/A  {'Gene-ref': {'Gene-ref_locus': 'RSPO1', 'Gene...
76     N/A  {'Gene-ref': {'Gene-ref_locus': 'MET', 'Gene-r...
77     N/A  {'Gene-ref': {'Gene-ref_locus': 'CXCL12', 'Gen...
78     N/A  {'Gene-ref': {'Gene-ref_locus': 'TGFA', 'Gene-...

[79 rows x 2 columns]
